# __Static Portfolio - 311 Service Requests in Chicago__

### Data Preprocessing

- [Preprocessing of data](https://colab.research.google.com/drive/1l6l6Lp4xRcRU9rqpS7WInSlaStZVaTgb)

### Import Dependencies

In [0]:
!apt install python3-rtree
!pip install git+git://github.com/geopandas/geopandas.git -U
!pip3 install vega altair -U

from google.colab import drive
import altair as alt
import pandas as pd
import geopandas as gpd

drive.mount('/content/gdrive')
alt.data_transformers.disable_max_rows()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-vyhzb4ki
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-vyhzb4ki
  Created wheel for geopandas: filename=geopandas-0.6.1+39.gfcbc6ce-py2.py3-none-any.whl size=922041 sha256=c416563eb003ba04fba8215e91e6b85676ac8f3de074c463ad54afff2dbe8a6b
  Stored in directory: /tmp/pip-ephem-wheel-cache-f7odotwh/wheels/91/24/71/376c9c67192694168352afcccc2d264248f7e2cc6192997186
Successfully built geopandas
  Found existing installation: geopandas 0.6.1+39.gfcbc6ce
    Uninstalling geopandas-0.6.1+39.gfcbc6ce:
      Successfully uninstalled g

Requirement already up-to-date: vega in /usr/local/lib/python3.6/dist-packages (2.6.0)
Requirement already up-to-date: altair in /usr/local/lib/python3.6/dist-packages (4.0.1)
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


DataTransformerRegistry.enable('default')

### Helper Functions

In [0]:
def create_count_df_by_time(df, time_col):

    df = df[[time_col, "CREATED_YEAR", "STATUS"]]\
        .groupby([time_col, "CREATED_YEAR"])\
        .STATUS.agg("count")\
        .reset_index(name="COUNT")
    df["COUNT"] = df["COUNT"].div(1000)
    return df

def create_count_df_by_year(dfs, time_col, verbose=False):
    
    merged_df = create_count_df_by_time(dfs[0], time_col)
    for df in dfs[1:]:
        df = create_count_df_by_time(df, time_col)
        merged_df = merged_df.append(df, ignore_index=True)
    if verbose:
        print(merged_df.sample(n=5))
    return merged_df

### Theme

In [0]:
def theme_311():
    # Theme Template from https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602

    markColor = "#1696d2"
    axisColor = "#000000"
    backgroundColor = "#f4f3f3"
    font = "Roboto"
    labelFont = "Roboto"
    sourceFont = "Roboto"
    gridColor = "#d6d4d4"
    main_palette = [
        "#76b7b2", "#f28e2b", "#e15759",
        "#4e79a7", "#59a14f", "#edc948",
        "#b07aa1", "#ff9da7", "#9c755f",
        "#bab0ac"
    ]
    sequential_palette = [
        "#e1f5fe", "#b3e5fc", "#81d4fa", 
        "#4fc3f7", "#29b6f6", "#03a9f4",
        "#039be5", "#0288d1", "#0277bd",
        "#01579b"
    ]
    return {
        "config": {
            "title": {
                "anchor": "start",
                "fontSize": 24,
                "font": font,
                "fontColor": "#000000"
            },
            "subtitle": {
                "font": font,
                "fontSize": 22,
                "fontColor": "#838383",
                "subtitlePadding": 10
            },
            "caption": {
                "font": font,
                "fontSize": 22,
                "fontColor": "#000000"
            },
            "axisX": {
               "domain": True,
               "domainColor": axisColor,
               "domainWidth": 1,
               "grid": False,
               "labelFontSize": 13,
               "labelFont": labelFont,
               "labelAngle": 0,
               "tickColor": axisColor,
               "tickSize": 5,
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "title": "",
           },
           "axisY": {
               "domain": False,
               "grid": True,
               "gridColor": gridColor,
               "gridWidth": 1,
               "labelFontSize": 12,
               "labelFont": labelFont,
               "labelPadding": 8,
               "ticks": False,
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "titleAngle": 270,
           },
           "background": backgroundColor,
           "legend": {
               "labelFontSize": 12,
               "labelFont": labelFont,
               "symbolSize": 300,
               "symbolType": "circle",
               "titleFontSize": 15,
               "titlePadding": 10,
               "titleFont": font,
               "orient": "right",
               "offset": 20,
           },
           "range": {
               "category": main_palette,
               "diverging": sequential_palette,
           },
           "geoshape": {
               "category": main_palette,
               "diverging": sequential_palette,
           },
           "area": {
               "category": main_palette,
               "fill": markColor,
               "opacity": 0.7,
           },
           "line": {
               "color": markColor,
               "stroke": markColor,
               "strokewidth": 5,
           },
           "point": {
               "filled": True,
           },
           "circle": {
               "color": "#4e79a7",
               "size": 13,
           },
           "text": {
               "font": sourceFont,
               "color": markColor,
               "fontSize": 11,
               "align": "right",
               "fontWeight": 400,
               "size": 11,
           }, 
           "bar": {
                "size": 40,
                "binSpacing": 1,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": markColor,
                "stroke": False,
                "opacity": 0.7
            }, 
       },
    }

alt.themes.register("theme_311", theme_311)
alt.themes.enable("theme_311")

ThemeRegistry.enable('theme_311')

### __1. Load the dataset__

In [0]:
# Load the dataset for 2018
data_dir_2018 = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311_2018.csv"
df_2018 = pd.read_csv(data_dir_2018)
# Load the dataset for 2019
data_dir_2019 = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311_2019.csv"
df_2019 = pd.read_csv(data_dir_2019)
# Load the entire dataset
data_dir = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311.csv"
df = pd.read_csv(data_dir)

In [0]:
day_source = create_count_df_by_year([df_2018, df_2019], "CREATED_DAY_OF_WEEK")
day_dict = {
    1: "Sunday",
    2: "Monday",
    3: "Tuesday",
    4: "Wednesday",
    5: "Thursday",
    6: "Friday",
    7: "Saturday",
}
month_dict = {
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}
day_source["CREATED_DAY_OF_WEEK"] = day_source["CREATED_DAY_OF_WEEK"].apply(lambda x: day_dict[x])
month_source = create_count_df_by_year([df_2018, df_2019], "CREATED_MONTH")
month_source["MONTH"] = month_source["CREATED_MONTH"].apply(lambda x: month_dict[x])

### __2. The Number of 311 Service Requests in Chicago in 2017 and 2018__

#### Code for data visualization

In [0]:
month_count = alt.Chart(month_source).mark_bar(opacity=0.7, size=45).encode(
    x=alt.X(
        'CREATED_YEAR:N',
        axis=alt.Axis(labelAngle=0),
        title=""
        ),
    y=alt.Y(
        'COUNT:Q',
        title="The Number of 311 Requests (in 1000)",
        ),
    color=alt.Color(
        "CREATED_YEAR:N",
        legend=alt.Legend(title=""),
        ),
    column=alt.Column(
        'MONTH:N',
        sort=['July','August','September','October','November','December']
        )
).properties(
    title={
      "text": ["The Number of 311 Service Requests in 2018 and 2019", "by Month (July - December) and Day of Week"], 
      "subtitle": ["Chicago Data Portal - 311 Service Requests"],
      "subtitleColor": "grey"
    },
    width=90,
    height=300
)

In [0]:
day_count = alt.Chart(day_source).mark_area().encode(
    x=alt.X(
        "CREATED_DAY_OF_WEEK:N",
        axis=alt.Axis(labelAngle=0),
        title="Day of Week",
        sort=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
        ),
    y=alt.Y(
        "COUNT:Q",
        title="The Number of 311 Requests (in 1000)"
        ),
    color=alt.Color(
        "CREATED_YEAR:N",
        legend=alt.Legend(title=""),
        )
    ).properties(
        width=650,
        height=300
    )
count = alt.vconcat(month_count, day_count)

#### *Chart #1*

In [0]:
count

alt.VConcatChart(...)

The charts above, created from [Chicago Data Portal](https://data.cityofchicago.org/Service-Requests/311-Service-Requests/v6vf-nfxy), show *the number of 311 service requests made in 2018 and 2019 by month and day of week*. Notice that this chart only shows the number of 311 requests from July to December. This is because the dataset is only available from July to December for 2018 (*the dataset was created with the creation of the new 311 system and only partial legacy data is available*), so the data from January to June has been redacted in the chart.

These two charts show that there were consistently __more 311 service requests in 2019 than in 2018__, which is consistent with the City of Chicago's claim that 311 became more accessible through its recent upgrade in late 2018/early 2019. Another interesting observation is that there are less 311 service requests during weekends than during weekdays, a pattern that one would expect to find in crime data, which can also be found in Chicago Data Portal.

### __3. The Number of 311 Service Requests by Owner Department in Chicago in 2017 and 2018__

#### Code for data visualization

In [0]:
dept_count = df_2018.groupby(['OWNER_DEPARTMENT', "CREATED_YEAR"]).size().sort_values(ascending=False).reset_index(name='COUNT')
top_10_type_2018 = dept_count.nlargest(8, 'COUNT')
dept_count = df_2019.groupby(['OWNER_DEPARTMENT', "CREATED_YEAR"]).size().sort_values(ascending=False).reset_index(name='COUNT')
top_10_type_2019 = dept_count.nlargest(8, 'COUNT')
top_10_type = top_10_type_2018.append(top_10_type_2019, ignore_index=True)
dept = [
        "Streets and Sanitation",
        "CDOT - Department of Transportation",
        "DWM - Department of Water Management",
        "DOB - Buildings",
        "Animal Care and Control",
        "BACP - Business Affairs and Consumer Protection",
        "311 City Services"
        ]
top_10_type = top_10_type[top_10_type.OWNER_DEPARTMENT.isin(dept)].reset_index(drop=True)
dept_dict = {
    "Streets and Sanitation": "Streets and Sanitation",
    "CDOT - Department of Transportation": "Transportation",
    "DWM - Department of Water Management": "Water Management",
    "DOB - Buildings": "Buildings",
    "Animal Care and Control": "Animal Care / Control",
    "BACP - Business Affairs and Consumer Protection": "Business Affairs",
    "311 City Services": "311 City Services"
    }
top_10_type["OWNER_DEPARTMENT"] = top_10_type["OWNER_DEPARTMENT"].apply(lambda x: dept_dict[x])
top_10_type["COUNT"] = top_10_type["COUNT"] / 1000

In [0]:
dept_count_chart = alt.Chart(top_10_type).mark_bar(opacity=0.7, size=45).encode(
    x=alt.X(
        'CREATED_YEAR:N',
        axis=alt.Axis(labelAngle=0),
        title=""
        ),
    y=alt.Y(
        'COUNT:Q',
        title="The Number of 311 Requests (in 1000)",
        ),
    color=alt.Color(
        "CREATED_YEAR:N",
        legend=alt.Legend(title="Year"),
        ),
    column=alt.Column(
        'OWNER_DEPARTMENT:N',
        title="Owner Department"
        )
).properties(
    title={
      "text": ["The Number of 311 Service Requests in 2018 and 2019", "by Owner Department"], 
      "subtitle": ["Chicago Data Portal - 311 Service Requests"],
      "subtitleColor": "grey"
    },
    width=100,
    height=300
)


#### *Chart #2*

In [0]:
dept_count_chart

alt.Chart(...)

From this chart, we can see that the huge increase in the number of 311 service requests from 2018 to 2019 is due to the increase in 311 City Services. Looking at the dataset, the only service request type that dealt by 311 City Services Department is `311 Information Calls`. This observation makes sense, since the development of the new 311 system should have given easier access to the 311 service, so it makes sense that people would use the new 311 system to request simple information.

### __4. The Percentage of 311 Service Request Status in Chicago in 2017 and 2018__

#### Code for chart

In [0]:
status_df_2018 = df_2018['STATUS'].value_counts(normalize=True).reset_index(name='COUNT')
status_df_2018["YEAR"] = 2018
status_df_2019 = df_2019['STATUS'].value_counts(normalize=True).reset_index(name='COUNT')
status_df_2019["YEAR"] = 2019
status_df = status_df_2018.append(status_df_2019, ignore_index=True)
bars = alt.Chart(status_df).mark_bar(opacity=0.7, size=100).encode(
    x=alt.X(
        'sum(COUNT):Q',
        stack="normalize",
        title="Percentage of Status"
        ),
    y=alt.Y(
        'YEAR:N',
        title="Year"
        ),
    color=alt.Color(
        'index',
        legend=alt.Legend(title="Status", orient='right')
    )          
).properties(
    title={
      "text": ["The Percentage of 311 Service Request Status in 2018 and 2019", "(Second-half of 2018 and 2019)"], 
      "subtitle": ["Chicago Data Portal - 311 Service Requests"],
      "subtitleColor": "grey"
    },
    width=600,
    height=300
)
average_days = pd.DataFrame({
    'Year': [2018, 2019],
    'Average': [
                df_2018[df_2018['STATUS']=="Completed"]["DAYS_TAKEN"].mean(),
                df_2019[df_2019['STATUS']=="Completed"]["DAYS_TAKEN"].mean()
                ]
})

days = alt.Chart(average_days).mark_bar(opacity=0.7, color='#f28e2b').encode(
    x=alt.X('Year:N', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Average', title="Average Number of Days")
).properties(
    width=100,
    height= 300
)

#### *Chart #3*

In [0]:
bars | days

alt.HConcatChart(...)

Just comparing the current status of 311 service requests of 2018 and 2019 (July - December), higher percentage of requests were completed in 2018 than in 2019. This is to be expected, since there were more time to complete the request. However, it is interesting that the average number of days taken to complete a 311 request in 2018 is around 28 days whereas it only takes 6 days on average to complete a 311 request in 2019.

### __5. Relationship between Budget and 311 Service Requests__

#### Code for Data Visualization

In [0]:
budget_dir = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/budget/2019_Budget_Ordinance.csv"
budget_df = pd.read_csv(budget_dir)
dept = [
        "DSS",
        "CDOT",
        "DWM",
        "BUILDINGS",
        "ANIMAL CARE / CONTROL",
]
dept_dict = {
    "DSS": "Streets and Sanitation",
    "CDOT": "Transportation",
    "DWM": "Water Management",
    "BUILDINGS": "Buildings",
    "ANIMAL CARE / CONTROL": "Animal Care / Control",
}
budget_df = budget_df[budget_df["DEPARTMENT DESCRIPTION"].isin(dept)].reset_index(drop=True)
budget_df["DEPARTMENT DESCRIPTION"] = budget_df["DEPARTMENT DESCRIPTION"].apply(lambda x: dept_dict[x])
budget_df = budget_df.groupby(['DEPARTMENT DESCRIPTION'])['2019 ORDINANCE (AMOUNT $)'].agg('sum').reset_index()
budget_df["2019 ORDINANCE (AMOUNT $)"] = budget_df["2019 ORDINANCE (AMOUNT $)"]/1000000
budget_df = budget_df.rename(columns={"DEPARTMENT DESCRIPTION": "OWNER_DEPARTMENT"})
budget_df = budget_df.merge(top_10_type[top_10_type["CREATED_YEAR"]==2019], on='OWNER_DEPARTMENT', how='left')

In [0]:
points = alt.Chart(budget_df).mark_point(size=180).encode(
    x=alt.X('COUNT:Q', title="Number of 311 Requests (in 1000)"),
    y=alt.Y('2019 ORDINANCE (AMOUNT $):Q', title="2019 Budget Ordinance (in $1000000)"),
    color='OWNER_DEPARTMENT'
).properties(
    title={
      "text": ["Scatterplot for", "the Number of 311 Requests", "and the Budget Ordinance of Chicago (2019)"], 
      "subtitle": ["Chicago Data Portal - 311 Service Requests", "Chicago Data Portal - Budget - 2019 Budget Ordinance - Appropriations"],
      "subtitleColor": "grey"
    },
    width=500,
    height=500
)

text = points.mark_text(
    align='left',
    baseline='middle',
    dx=15
).encode(
    text='OWNER_DEPARTMENT'
)

scatterplot = points + text

#### *Chart #4*

In [0]:
scatterplot

alt.LayerChart(...)

Using the data from [Chicago Data Portal](https://data.cityofchicago.org/Administration-Finance/Budget-2019-Budget-Ordinance-Appropriations/h9rt-tsn7), I had hoped to see some relationship between the amount of budget allocated to various departments and the number of 311 requests handled by them. Unfortunately, from the departments that handle the most number of 311 service requests, it was difficult to notice any distinct pattern. In hindsight, I should have looked deeper into the budget allocation within each department, since not all requests cost the same. For instance, the requests that the department of Transportation handle may take more resources than the requests that are dealt by Animal Care / Control.

### __6. Potholes__

The department of Streets and Sanitations receives many 311 service requests regarding potholes. Let us see if there are any geographical patterns regarding pothole-related requests.


#### Code for data visualization

In [0]:
census_json = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/census_tract_boundary/tracts.geojson"
gdf = gpd.read_file(census_json)
gdf.crs = "epsg:4326"
data_geojson = alt.InlineData(values=gdf.to_json(), format=alt.DataFormat(property='features', type='json'))

pothole_filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_311_pothole.csv"
pothole_df = pd.read_csv(pothole_filename)
pothole_df = pothole_df.sample(n=10000)

chicago_map = alt.Chart(data_geojson).mark_geoshape(
    fill='lightgray',
    stroke='white',
    strokeWidth=0.5
    ).properties(
        width=600,
        height=600,
        title={
        "text": ["Scatter Plot Map of", "the Number of 311 Requests for Potholes", "per Census Tract in Chicago (2018-2020)"], 
        "subtitle": ["Chicago Data Portal - 311 Service Requests", "Chicago Data Portal - Boundaries - Census Tracts"],
        "subtitleColor": "grey",
        "color": "black"
        }
     ).encode()

points = alt.Chart(pothole_df).mark_circle(
    size=10,
    baseline='top'
).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q',
    tooltip=['SR_NUMBER', 'ZIP_CODE', 'WARD']
)

pothole_map = chicago_map + points

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### *Chart #5*

In [0]:
pothole_map

This chart shows that many pothole-related 311 requests are focused in the northern part of Chicago such as River North, Lincoln Park, or Logan Park. However, it is not clear from this chart if potholes are simply more frequent in these parts of the city or there are more 311 requests overall.

### __7. Census Tracts of Chicago__

#### Code for Data Visualization

In [0]:
census_json = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/census_tract_boundary/tracts.geojson"
tracts_311_count_filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_count.csv"
tracts_count = pd.read_csv(tracts_311_count_filename)
gdf = gpd.read_file(census_json)
gdf = gdf.merge(tracts_count, on='namelsad10', how='left')
gdf.crs = "epsg:4326"
data_geojson = alt.InlineData(values=gdf.to_json(), format=alt.DataFormat(property='features', type='json')) 

map_chart = alt.Chart(data_geojson).mark_geoshape(
    stroke='black',
    strokeWidth=0.5,
    ).encode(
        color = alt.Color(
            'properties.color:N',
            legend=alt.Legend(
                title="Number of Requests",
                titleFontSize = 11,
                columns=3,
                labelFontSize=0,
                symbolSize=1000,
                columnPadding=0,
                rowPadding=0,
                labelPadding=0
                ),
            scale=alt.Scale(
                domain=["#64acbe", "#b0d5df", "#e8e8e8", "#627f8c", "#ad9ea5", "#e4acac", "#574249", "#985356", "#c85a5a"],
                range=["#73ae80", "#b8d6be", "#cccccc", "#5a9178", "#90b2b3", "#b5c0da", "#2a5a5b", "#567994", "#6c83b5"]
                )
        ),
        opacity=alt.value(0.8),
    ).properties(
        title={
        "text": ["Bivariate Choropleth Map of the Number of 311 Requests", "per Census Tract in Chicago (2018-2020)"], 
        "subtitle": ["Chicago Data Portal - 311 Service Requests", "Chicago Data Portal - Boundaries - Census Tracts"],
        "subtitleColor": "grey",
        "color": "black"
        },
        projection={"type":'mercator'},
        width = 600,
        height = 600
)


#### *Chart #6*

In [0]:
map_chart

__Reading the bivariate choropleth map__

As you can see on the right side of the Chicago map, there are a total of 9 different color on this map. The number of 311 service requests is represented by *x-axis* (i.e., the right, the more), and the average household income level is represented by the *y-axis* (i.e., the up, the more).


__Findings__

Using the `geojson` data from [Chicago Data Portal](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Census-Tracts-2010/5jrd-6zik) and [US Census Data](https://data.census.gov/cedsci/table?q=Income%20and%20Earnings&g=0500000US17031.140000&t=Income%20and%20Earnings&hidePreview=true&table=S1901&tid=ACSST5Y2018.S1901&lastDisplayedRow=0&vintage=2017&mode=), I charted a bivariate choropleth map of using the number of 311 requests and the average household income level by census tract in Chicago. There seem to be some visual patterns here. First of all, there seem to be a clear divide in the average household income level where the north and east of Chicago is generally more wealthy than other parts of Chicago. Furthermore, areas above the downtown along the river (e.g., River North, West Lakeview) seem to be high in both average household income level and the number of 311 requests. 

### __8. What is wrong with Census Tract 8382?__

Let us look at the table below. It looks like __Census Tract 8382__ has an outstanding number of 311 service requests. By just looking at this statistics, one might think that this part of the city has some serious issues.

In [0]:
tracts_count.sort_values('Number_of_Requests', ascending=False).head(5)

,namelsad10,Number_of_Requests,Household_Income,color
746,Census Tract 8382,610687,17817,#574249
752,Census Tract 8391,6573,22076,#574249
405,Census Tract 4910,5807,6707,#c85a5a
159,Census Tract 2315,5665,8036,#c85a5a
436,Census Tract 5302,5391,8114,#c85a5a


However, this might not be the case. We already know from one of the charts that a high number of the 311 service requests are directed to __311 City Services__. It is possible that the location of all __311 INFORMATION ONLY CALLS__ that are directed to __311 City Services__ are simply coded to [__311 Operations__](https://goo.gl/maps/5znZXVSsybrMcqxw7). Indeed, if you look at Google Maps, you can see that __311 Operations__ is located in Census Tract 8382.

#### Code for Data Visualization

In [0]:
census_json = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/census_tract_boundary/tracts.geojson"
gdf = gpd.read_file(census_json)
gdf.crs = "epsg:4326"
data_geojson = alt.InlineData(values=gdf.to_json(), format=alt.DataFormat(property='features', type='json'))

call_filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_311_call.csv"
call_df = pd.read_csv(call_filename)
call_df = call_df.sample(n=10000)

census_map = alt.Chart(data_geojson).mark_geoshape(
    fill='lightgray',
    stroke='white',
    strokeWidth=0.5
    ).properties(
        width=800,
        height=800,
        title={
        "text": ["Location of 311 Information-only Calls", "by Census Tract in Chicago (2018-2020)"], 
        "subtitle": ["Chicago Data Portal - 311 Service Requests", "Chicago Data Portal - Boundaries - Census Tracts"],
        "subtitleColor": "grey",
        "color": "black"
        }
     ).encode()

points = alt.Chart(call_df).transform_aggregate(
    latitude='mean(LATITUDE)',
    longitude='mean(LONGITUDE)',
    count='count(SR_NUMBER)',
    groupby=['namelsad10']
).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('count:Q', title='Number of 311 Requests'),
    tooltip=['namelsad10:N','count:Q']
).properties(
    title='Number of 311 Requests'
)

info_311 = census_map + points

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### *Chart # 7*

In [0]:
info_311

Because of the limited Colab resources, it was not possible to plot all of the __311 INFORMATION ONLY CALLS__. So I plotted random 30000 data points on the map. As you can see above, the overwhelming majority of them are located in Census Tract 8382. This supports our hypothesis that Census Tract 8382 is not a big mess!